# Create db with Python

In [1]:
import numpy as np
#import descartes
import pandas as pd
import matplotlib.pyplot as pp
import geopandas as gpd
#%matplotlib inline
%matplotlib widget
import contextily as ctx
import geoplot as gpt
import mapclassify
from math import radians, cos,sin,asin,sqrt
pd.set_option('display.max_columns',30)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridspecLayout
from ipywidgets import TwoByTwoLayout
import psycopg2
from psycopg2 import Error
from shapely.wkt import dumps,loads

In [2]:
# Medical facility data (name,coordinate)
medical_df=pd.read_csv('Medical_service_info_corrected.csv')
# Population data (district) & Population data (chiefdom, not necessary is population data is detailed)
district_population_df=pd.read_csv('Population_info.csv')
chiefdom_population_df=pd.read_csv('Chiefdom data_Bo and Kenema.csv')
# Shape file (district) & Shape file (chiefdom)
district_shape_df=gpd.read_file('sle_admbnda_adm2_1m_gov_ocha/')
# chiefdom_shape_df=gpd.read_file('sle_admbnda_adm4_1m_gov_ocha/')
# EEZ boundaries file
eez_shape_df=gpd.read_file('World_EEZ_v11_20191118/')
eez_shape_df=eez_shape_df[eez_shape_df['TERRITORY1']=='Sierra Leone']
# City medical facility data (Not necessary if medical facility data is accurate and complete)
# Hospitals in Bo and Kenema
hospital_partial_df=pd.read_csv('Site_location_programing.csv')

In [376]:
def change_coordinates(facility_name,x,y):
    """
    change facility coordinates in medical_df
    """
    medical_gdf.loc[medical_gdf.name==facility_name,'latitude']=x
    medical_gdf.loc[medical_gdf.name==facility_name,'longitude']=y
    facility=medical_gdf.loc[medical_gdf.name==facility_name]
    facility=gpd.GeoDataFrame(facility,geometry=gpd.points_from_xy(facility.longitude,facility.latitude))

In [377]:
# Create district info dataframe and GeoDataFrames
district_info_df=pd.merge(district_shape_df,district_population_df, left_on=['admin2RefN'],right_on=['district'])
medical_gdf=gpd.GeoDataFrame(medical_df,geometry=gpd.points_from_xy(medical_df.longitude,medical_df.latitude))

# Many coordinates are not accurate in this 'Medical_service_info_corrected.csv' file
change_coordinates('Ola During (Epi)',8.490446, -13.216735)

# Create population calculation dataframe 
population_cal_df=pd.DataFrame(chiefdom_population_df.groupby('section').latitude.mean()).reset_index()
population_cal_df['longitude']=chiefdom_population_df.groupby('section').longitude.mean().values
population_cal_df['population']=chiefdom_population_df.groupby('section').total.sum().values

In [ ]:
hospital_gdf=gpd.GeoDataFrame(hospital_partial_df,geometry=gpd.points_from_xy(hospital_partial_df.longitude,hospital_partial_df.latitude))
chiefdom_population_df=gpd.GeoDataFrame(chiefdom_population_df,geometry=gpd.points_from_xy(chiefdom_population_df.longitude,chiefdom_population_df.latitude))

## Medical_db

In [350]:
medical_g=medical_gdf[['district','name','address','facility_type','Population','geometry']]
medical_g.head(10)

,district,name,address,facility_type,Population,geometry
0,Bo,Bo Gov. Hospital,Bo,HOSPITAL,574918,POINT (-11.72079 7.96190)
1,Bo,Children Hospital,Bo Children's Hospital,HOSPITAL,574918,POINT (-11.71897 7.96102)
2,Bo,Boima Hospital,Sowa Clinic,HOSPITAL,574918,POINT (-11.71945 7.96212)
3,Bo,Gila'S Hospital,Gila's Hospital,HOSPITAL,574918,POINT (-11.71992 7.96119)
4,Bo,Kindoya Hospital,Kindoya Hospital,HOSPITAL,574918,POINT (-11.73899 7.96009)
5,Bo,Mercy Hospital,Kukua Kulanda Town,HOSPITAL,574918,POINT (-11.71884 7.96176)
6,Bo,Serabu Catholic Hospital,"Serabu town, bumpeh ngao, bo ...",HOSPITAL,574918,POINT (-11.71900 7.96328)
7,Bo,Agape Clinic,Kakua,CHC,574918,POINT (-11.71983 7.96373)
8,Bo,Baomahun,Baomahun,CHC,574918,POINT (-11.66755 8.41651)
9,Bo,Bumpe,BUMPEH- BUMPE NGAO,CHC,574918,POINT (-11.71809 7.96311)


In [353]:
medical_g.index.values

array([   0,    1,    2, ..., 1326, 1327, 1328])

In [349]:
#medical_g.to_csv('medical.csv',index=False)

In [351]:
conn=psycopg2.connect(host="localhost",database='sierra_geo',user='postgres')
cursor=conn.cursor()

In [347]:
cursor.execute("DROP TABLE IF EXISTS medical")
cursor.execute("CREATE TABLE medical (id SERIAL PRIMARY KEY, district VARCHAR, name VARCHAR, address VARCHAR, type VARCHAR, population INTEGER, point GEOMETRY)")
conn.commit()

In [341]:
cursor.execute("DELETE FROM medical")
for i in medical_g.index.values:
    #cursor.execute(sql,(medical_g.iloc[i].district))
    cursor.execute("INSERT INTO medical (district,type,population) VALUES ('{}','{}','{}')".format(medical_g.iloc[i].district,medical_g.iloc[i].facility_type,medical_g.iloc[i].Population))
        
conn.commit()

In [352]:
#cursor.execute("DELETE FROM medical")
#cursor.execute("INSERT INTO medical (geometry) VALUES (ST_AsEWKB('{}'))".format(medical_g.iloc[0].geometry.wkt))
for i in medical_g.index.values:
    cursor.execute("UPDATE medical SET point='{}' WHERE id={};".format(medical_g.iloc[i].geometry.wkt,i+1))
        
conn.commit()

## District_db

In [357]:
district_info_df.head(5)

,OBJECTID,admin2Name,admin2Pcod,admin2RefN,admin2AltN,admin2Al_1,admin1Name,admin1Pcod,admin0Name,admin0Pcod,date,validOn,ValidTo,Shape_Leng,Shape_Area,geometry,district,Population
0,1,Pujehun,SL0304,Pujehun,None,None,Southern,SL03,Sierra Leone,SL,2016-08-01,2016-10-17,None,3.518763,0.342539,"POLYGON ((-11.49263 7.62594, -11.49128 7.62408...",Pujehun,346461
1,2,Port Loko,SL0204,Port Loko,None,None,Northern,SL02,Sierra Leone,SL,2016-08-01,2016-10-17,None,6.465113,0.491646,"MULTIPOLYGON (((-13.13135 8.53712, -13.13194 8...",Port Loko,530865
2,3,Bonthe,SL0302,Bonthe,None,None,Southern,SL03,Sierra Leone,SL,2016-08-01,2016-10-17,None,5.983154,0.293946,"MULTIPOLYGON (((-12.56262 7.37628, -12.56272 7...",Bonthe,200781
3,4,Bo,SL0301,Bo,None,None,Southern,SL03,Sierra Leone,SL,2016-08-01,2016-10-17,None,4.392560,0.448247,"POLYGON ((-11.59006 8.48235, -11.59006 8.46775...",Bo,574918
4,5,Kambia,SL0202,Kambia,None,None,Northern,SL02,Sierra Leone,SL,2016-08-01,2016-10-17,None,4.572489,0.248748,"MULTIPOLYGON (((-13.26358 8.85189, -13.26458 8...",Kambia,346213


In [367]:
district_df=district_info_df[['admin2Name','admin1Name','geometry','Population']]
district_df

,admin2Name,admin1Name,geometry,Population
0,Pujehun,Southern,"POLYGON ((-11.49263 7.62594, -11.49128 7.62408...",346461
1,Port Loko,Northern,"MULTIPOLYGON (((-13.13135 8.53712, -13.13194 8...",530865
2,Bonthe,Southern,"MULTIPOLYGON (((-12.56262 7.37628, -12.56272 7...",200781
3,Bo,Southern,"POLYGON ((-11.59006 8.48235, -11.59006 8.46775...",574918
4,Kambia,Northern,"MULTIPOLYGON (((-13.26358 8.85189, -13.26458 8...",346213
5,Kailahun,Eastern,"POLYGON ((-10.31537 8.50033, -10.31537 8.49692...",527248
6,Koinadugu,Northern,"POLYGON ((-11.18488 9.96709, -11.17875 9.96479...",204264
7,Bombali,Northern,"POLYGON ((-11.90923 9.94417, -11.90389 9.93830...",422960
8,Moyamba,Southern,"MULTIPOLYGON (((-12.99093 7.90855, -12.99121 7...",319148
9,Kenema,Eastern,"POLYGON ((-11.40796 8.55276, -11.40227 8.54939...",609891


In [368]:
conn=psycopg2.connect(database='sierra_geo',host='localhost',user='postgres')
cursor=conn.cursor()

In [369]:
cursor.execute("DROP TABLE IF EXISTS district")
cursor.execute("CREATE TABLE district (id SERIAL PRIMARY KEY, district_name VARCHAR, region_name VARCHAR, population INTEGER, polygon GEOMETRY)")
conn.commit()

In [370]:
#cursor.execute('DELETE FROM district')
for i in district_df.index.values:
    cursor.execute("INSERT INTO district(district_name,region_name,population,polygon) VALUES('{}','{}','{}','{}')".format(district_df.iloc[i].admin2Name,
                                                                                                                        district_df.iloc[i].admin1Name,
                                                                                                                        district_df.iloc[i].Population,
                                                                                                                       district_df.iloc[i].geometry))
conn.commit()

## Population_cal_db

In [378]:
population_cal_df.head(2)

,section,latitude,longitude,population
0,Baimba,8.069610,-11.666394,2657
1,Bainyawa,7.699136,-11.740132,2238


In [380]:
population_cal_df.to_csv('pop_cal.csv',index=False)

## Chiefdom_population_df

In [31]:
#chiefdom_population_df=chiefdom_population_df[['district','chiefdom','section','total','latitude','longitude','geometry']]
chiefdom_population_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   district   2149 non-null   object  
 1   chiefdom   2149 non-null   object  
 2   section    2149 non-null   object  
 3   total      2149 non-null   int64   
 4   latitude   2149 non-null   float64 
 5   longitude  2149 non-null   float64 
 6   geometry   2149 non-null   geometry
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 117.6+ KB


In [13]:
conn=psycopg2.connect(host='localhost',database='sierra_geo',user='postgres')
cursor=conn.cursor()

In [9]:
cursor.execute('DROP TABLE IF EXISTS chiefdom_population')
cursor.execute('CREATE TABLE chiefdom_population (id SERIAL PRIMARY KEY, district VARCHAR, chiefdom VARCHAR,section VARCHAR,total INTEGER,latitude NUMERIC,longitude NUMERIC,point GEOMETRY)')
conn.commit()

In [14]:
#cursor.execute('DELETE FROM chiefdom_population')
for i in chiefdom_population_df.index.values:
    cursor.execute("INSERT INTO chiefdom_population(district,chiefdom,section,total,latitude,longitude,point) VALUES('{}','{}','{}','{}','{}','{}','{}')".format(chiefdom_population_df.iloc[i].district,
                                                                                                                       chiefdom_population_df.iloc[i].chiefdom,
                                                                                                                        chiefdom_population_df.iloc[i].section,
                                                                                                                       chiefdom_population_df.iloc[i].total,
                                                                                                                       chiefdom_population_df.iloc[i].latitude,
                                                                                                                       chiefdom_population_df.iloc[i].longitude,
                                                                                                                       chiefdom_population_df.iloc[i].geometry))
conn.commit()

In [15]:
conn.close()

## eez_shape_df

In [19]:
eez_shape_df.head(4)

,LINE_ID,LINE_NAME,LINE_TYPE,MRGID_SOV1,MRGID_TER1,TERRITORY1,SOVEREIGN1,MRGID_TER2,TERRITORY2,MRGID_SOV2,SOVEREIGN2,MRGID_EEZ1,EEZ1,MRGID_EEZ2,EEZ2,SOURCE1,URL1,SOURCE2,URL2,SOURCE3,URL3,ORIGIN,DOC_DATE,MRGID_JREG,JOINT_REG,LENGTH_KM,geometry
343,3658.0,Sierra Leone 200 NM,200 NM,2191.0,2191.0,Sierra Leone,Sierra Leone,0.0,None,0.0,None,8390.0,Sierra Leonian Exclusive Economic Zone,0.0,None,200NM limit,http://www.marineregions.org/eezmethodology.php,None,None,None,None,Buffer,2017-03-28,0.0,None,519.0,"LINESTRING (-13.56693 4.18726, -13.56757 4.187..."
777,3393.0,Sierra Leone - Guinea,Median line,2191.0,2191.0,Sierra Leone,Sierra Leone,2122.0,Guinea,2122.0,Guinea,8390.0,Sierra Leonian Exclusive Economic Zone,8472.0,Guinean Exclusive Economic Zone,VLIZ Median Line,http://www.marineregions.org/eezmethodology.php,None,None,None,None,Median,1995-09-28,0.0,None,400.0,"LINESTRING (-13.30213 9.03766, -13.31962 9.021..."
778,3394.0,Sierra Leone - Liberia,Median line,2191.0,2191.0,Sierra Leone,Sierra Leone,2105.0,Liberia,2105.0,Liberia,8390.0,Sierra Leonian Exclusive Economic Zone,8391.0,Liberian Exclusive Economic Zone,VLIZ Median Line,http://www.marineregions.org/eezmethodology.php,None,None,None,None,Median,1995-09-28,0.0,None,392.0,"MULTILINESTRING ((-13.56693 4.18726, -13.07960..."


In [24]:
conn=psycopg2.connect(host='localhost',database='sierra_geo',user='postgres')
cursor=conn.cursor()

In [21]:
cursor.execute('DROP TABLE IF EXISTS eez')
cursor.execute('CREATE TABLE eez (id SERIAL PRIMARY KEY, line_name VARCHAR, line GEOMETRY)')
conn.commit()

In [29]:
#cursor.execute('DELETE FROM chiefdom_population')
for i in range(3):
    cursor.execute("INSERT INTO eez(line_name,line) VALUES('{}','{}')".format(eez_shape_df.iloc[i].LINE_NAME, eez_shape_df.iloc[i].geometry
                                                                                                                        ))
conn.commit()

In [30]:
conn.close()

## hospital_partial_df

In [32]:
hospital_partial_df.head(2)

,Unnamed: 0,Name,district,address,facility_type,latitude,longitude,Note,geometry
0,0,Taihun Bongor Hospital,Bo,NaN,HOSPITAL,8.051035,-11.799861,City north (15km) *,POINT (-11.79986 8.05104)
1,1,Njala Kubuya Hospital,Bo,NaN,NaN,7.956097,-11.774338,City edge west *,POINT (-11.77434 7.95610)
